# ***Title Generation of Nepali News articles using Seq2Seq with Attention Mechanism***


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torchtext.vocab import vocab

import pandas as pd
import numpy as np

import random
import math
import time
import string
import re
from sklearn.model_selection import train_test_split

from collections import Counter
import spacy

## Loading Data from Drive

I have a dataset containing News articles, their titles along with their published date & id. Currently, I am trying to predict title of news given its contents, the id and published date are of no use to us. We will drop those columns but first, let's import the data.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/News Data/news_big.csv')

<ipython-input-3-0a6e96df579c>:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/News Data/news_big.csv')


In [4]:
data.head()

,id,date_published,title,news
0,208422p1,"असार ३०, २०७९ बिहिबार ११:०:०",सरकारप्रतिको गिर्दो जनविश्वास।,जनताको विश्वास र बैधतामा शासन गर्ने सरकारहरू ज...
1,159079p1,"असार ५, २०७७ शुक्रबार १२:४८:०",लद्दाखमा चीनको लक्ष्य।,अहिले लद्दाख संकटलाई लिएर भारतमा भएको बहस एउटै...
2,157801p1,"जेठ २३, २०७७ शुक्रबार ७:०:०",आगामी दिनमा बसाल्नुपर्ने आनीबानी।,पहिलाको जीवन सरल थियो। न मुखमा मास्क चाहिन्थ्य...
3,157795p1,"जेठ २३, २०७७ शुक्रबार ५:५९:०",आधुनिक अन्धकार युगमा पिरती।,जन्मनेबित्तिकै मुहारमा जातीयताको चिह्न लगाइद...
4,157001p1,"जेठ १४, २०७७ बुधबार १३:३२:०",शक्तिशाली देशको कोरोनाविरुद्ध कमजोर कदम।,"अमेरिकाका डोनाल्ड ट्रम्प, रुसका भ्लादिमिर पुटि..."


In [5]:
data.drop(["id", "date_published"], axis=1, inplace=True)
data.head()

,title,news
0,सरकारप्रतिको गिर्दो जनविश्वास।,जनताको विश्वास र बैधतामा शासन गर्ने सरकारहरू ज...
1,लद्दाखमा चीनको लक्ष्य।,अहिले लद्दाख संकटलाई लिएर भारतमा भएको बहस एउटै...
2,आगामी दिनमा बसाल्नुपर्ने आनीबानी।,पहिलाको जीवन सरल थियो। न मुखमा मास्क चाहिन्थ्य...
3,आधुनिक अन्धकार युगमा पिरती।,जन्मनेबित्तिकै मुहारमा जातीयताको चिह्न लगाइद...
4,शक्तिशाली देशको कोरोनाविरुद्ध कमजोर कदम।,"अमेरिकाका डोनाल्ड ट्रम्प, रुसका भ्लादिमिर पुटि..."


In [6]:
data.shape

(226432, 2)

In [7]:
data = data.sample(10000)
data.shape

(10000, 2)

In [8]:
data.iloc[25].title

'श्रीमानको जो आदेश।'

In [9]:
data.iloc[25].news

'  केही दिनयता सर्वोच्च अदालत पत्रकारहरूप्रति अचम्मलाग्दो गरी असहिष्णु देखिँदै आएको छ । सर्वोच्चको यो असहिष्णुताको रहस्य बुझिनसक्नु छ ।   सधैँजसो अदालतको आदेश, फैसलाबारे बुझ्न जाने पत्रकार समूहलाई अस्ति मंगलबार अस्वभाविक रूपमा इजलास प्रवेश गर्नै दिइएन । खुला इजलास भनिने उक्त स्थानमा पुग्न सर्वोच्चले पत्रकारका लागि निकै कठीन बनाइदियो ।   इजलास प्रवेश त परको कुरा, सर्वोच्च अदालत परिसर छिर्नै पत्रकारका लागि मंगलबार निकै कडाइ थियो । मूलगेटमै सुरक्षाकर्मीले पत्रकारहरुको कडा खानतलासी लिए र क्यामेरा त्यहीं छाड्न लगाएर बल्ल भित्र छिराए ।   त्यसरी बल्लतल्ल छिरेका पत्रकारहरु सर्वोच्चको भुइँतलाबाट माथि उक्लन त पाउँदै पाएनन् । सर्वोच्च आफैँले पत्रकारहरुलाई एउटा पास दिएको थियो, मंगलबारदेखि त सर्वोच्चले त्यसलाई खोटो नै बनाइदियो । सर्वोच्चकै छाप र प्रवक्ताको हस्ताक्षर भएको त्यो पास अब सर्वोच्चकै सहप्रवक्ता हेमन्त रावलका शब्दमा ‘प्रवक्ताको कार्यकक्षसम्म पुग्नमात्र’ प्रयोग गर्न सकिन्छ ।   ‘यो पासले तपाईँहरू यहाँसम्म आउन सक्नुहुन्छ,’ रावलले आफ्नै कार्यकक्षमा भने, ‘सुरक्षाकर्मीहरुले सुरक्षाको समस्या देखा

## Data Cleaning

In [10]:
def clean_text(text):
  #remove punctuations
  sentences = text.replace("\n", " ").split(u"।")
  f = [sentence.translate(str.maketrans('', '', string.punctuation)) for sentence in sentences]
  f = [x.strip() for x in f if len(x)>0]
  return " ".join(f)

In [11]:
# also there are unicode like \u200d and \xa0, let's clean them. We will also remove emojis in this function
def clean_text_unicode(text):
    # Replace \xa0 and \u200d with a blank space
    text = text.replace('\xa0', ' ').replace('\u200d', ' ').replace("’",' ').replace("‘",' ')


    emojis = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    text = re.sub(emojis, '', text)
    return text

In [12]:
cleaner = lambda x: clean_text_unicode(clean_text(x))

In [13]:
data.news = data.news.apply(cleaner)
data.title = data.title.apply(cleaner)

In [14]:
data.iloc[25].news

'केही दिनयता सर्वोच्च अदालत पत्रकारहरूप्रति अचम्मलाग्दो गरी असहिष्णु देखिँदै आएको छ सर्वोच्चको यो असहिष्णुताको रहस्य बुझिनसक्नु छ सधैँजसो अदालतको आदेश फैसलाबारे बुझ्न जाने पत्रकार समूहलाई अस्ति मंगलबार अस्वभाविक रूपमा इजलास प्रवेश गर्नै दिइएन खुला इजलास भनिने उक्त स्थानमा पुग्न सर्वोच्चले पत्रकारका लागि निकै कठीन बनाइदियो इजलास प्रवेश त परको कुरा सर्वोच्च अदालत परिसर छिर्नै पत्रकारका लागि मंगलबार निकै कडाइ थियो मूलगेटमै सुरक्षाकर्मीले पत्रकारहरुको कडा खानतलासी लिए र क्यामेरा त्यहीं छाड्न लगाएर बल्ल भित्र छिराए त्यसरी बल्लतल्ल छिरेका पत्रकारहरु सर्वोच्चको भुइँतलाबाट माथि उक्लन त पाउँदै पाएनन् सर्वोच्च आफैँले पत्रकारहरुलाई एउटा पास दिएको थियो मंगलबारदेखि त सर्वोच्चले त्यसलाई खोटो नै बनाइदियो सर्वोच्चकै छाप र प्रवक्ताको हस्ताक्षर भएको त्यो पास अब सर्वोच्चकै सहप्रवक्ता हेमन्त रावलका शब्दमा  प्रवक्ताको कार्यकक्षसम्म पुग्नमात्र  प्रयोग गर्न सकिन्छ  यो पासले तपाईँहरू यहाँसम्म आउन सक्नुहुन्छ  रावलले आफ्नै कार्यकक्षमा भने  सुरक्षाकर्मीहरुले सुरक्षाको समस्या देखाएपछि अब इजलास जान छुट्टै पास लिनु

**Looks like the text is almost clean. We will be ignoring other data impurities and will continue with other tasks.**

## Data Split, Tokenization and Vocabulary Creation

In [15]:
X_train, X_test, y_train, y_test = train_test_split(data["news"], data["title"], test_size=0.2, random_state=42)

In [16]:
max_words_news = X_train.apply(lambda x:x.split()).apply(len).max()
max_words_news

5151

In [17]:
max_words_title = y_train.apply(lambda x:x.split()).apply(len).max()
max_words_title

19

In [18]:
counter = Counter()
for X in X_train:
  counter.update(X.split())
for y in y_train:
  counter.update(y.split())

In [19]:
vocabs = vocab(counter, min_freq=10, specials=('<unk>', '<BOS>', '<EOS>', '<PAD>'))

In [20]:
vocabs.set_default_index(vocabs['<unk>'])

In [21]:
vocabs["जनताको"]

1539

In [22]:
class MyDataset(Dataset):
    def __init__(self, X,y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        news = self.X.iloc[idx]
        title = self.y.iloc[idx]

        news_indices = [vocabs[word] for word in news.split()]
        title_indices = [vocabs[word] for word in title.split()]

        if len(news_indices)<max_words_news:
            news_indices = news_indices + [1]*(max_words_news - len(news_indices))

        if len(title_indices)<max_words_title:
            title_indices = title_indices + [1]*(max_words_title - len(title_indices))
        return torch.tensor(news_indices), torch.tensor(title_indices)

In [23]:
train_dataset = MyDataset(X_train,y_train)
test_dataset = MyDataset(X_test,y_test)

In [24]:
train_loader = DataLoader(train_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [25]:
test_dataset[25][0], test_dataset[25][1]  # news and title in 25th index of train dataset

(tensor([586,   0,   0,  ...,   1,   1,   1]),
 tensor([    0, 16480,     9,     0,    22,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1]))

In [26]:
train_dataset[25]

(tensor([ 586, 2044, 1752,  ...,    1,    1,    1]),
 tensor([2047, 2048, 7328, 1466, 2056, 2057, 2058,  128,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1]))

In [27]:
# pytorch dataset object use index to return item, so need to reset non-continuoues index of divided dataset
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

## ***Building Model***

### ***Encoder***

In [28]:
class Encoder(nn.Module):
  def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout): # input_dim = source vocab size
    super().__init__()

    self.embedding = nn.Embedding(input_dim, emb_dim)

    self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional=True)

    self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim) # enc_hid_dim *2 because of bidirectional

    self.dropout = nn.Dropout(dropout)


  def forward(self, src):

    # src = [seq_len, batch_size]
    embeddings = self.dropout(self.embedding(src)) # -> [seq_len, batch_size, emb_dim]

    outputs, hidden = self.rnn(embeddings)

    #outputs = [src len, batch size, hid dim * num directions]
    #hidden = [n layers * num directions, batch size, hid dim]
    #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
    #outputs are always from the last layer

    hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

    #outputs = [src len, batch size, enc hid dim * 2]
    #hidden = [batch size, dec hid dim]

    return outputs, hidden

### ***Attention***

In [29]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()

        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)

    def forward(self, hidden, encoder_outputs):

        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]

        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]

        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]

        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2)))

        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)

        #attention= [batch size, src len]

        return F.softmax(attention, dim=1)

### ***Decoder***

In [30]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)

        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_outputs):

        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]

        input = input.unsqueeze(0)

        #input = [1, batch size]

        embedded = self.dropout(self.embedding(input))

        #embedded = [1, batch size, emb dim]

        a = self.attention(hidden, encoder_outputs)

        #a = [batch size, src len]

        a = a.unsqueeze(1)

        #a = [batch size, 1, src len]

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        #encoder_outputs = [batch size, src len, enc hid dim * 2]

        weighted = torch.bmm(a, encoder_outputs)

        #weighted = [batch size, 1, enc hid dim * 2]

        weighted = weighted.permute(1, 0, 2)

        #weighted = [1, batch size, enc hid dim * 2]
        print("embedded size:", embedded.size())
        print("weighted size:", weighted.size())
        rnn_input = torch.cat((embedded, weighted), dim = 2)

        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]

        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))

        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]

        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)

        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))

        #prediction = [batch size, output dim]

        return prediction, hidden.squeeze(0)

### ***Seq2Seq with Attention***

In [31]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio = 0.5):

        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time

        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)

        #first input to the decoder is the <sos> tokens
        input = trg[0,:]

        for t in range(1, trg_len):

            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)

            #place predictions in a tensor holding predictions for each token
            outputs[t] = output

            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio

            #get the highest predicted token from our predictions
            top1 = output.argmax(1)

            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### *Model Hyperparameters*

In [33]:
INPUT_DIM = len(vocabs)
OUTPUT_DIM = len(vocabs)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [34]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(19832, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(19832, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=19832, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [35]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 52,146,040 trainable parameters


In [36]:
optimizer = optim.Adam(model.parameters())

In [37]:
criterion = nn.CrossEntropyLoss(ignore_index = 1)

### Training Function



In [38]:
def train(model, iterator, optimizer, criterion, clip):

    model.train()

    epoch_loss = 0

    for i, batch in enumerate(iterator):

        src = batch[0].to(device)
        trg = batch[1].to(device)
        # print("DEVICE",src.get_device(),trg.get_device())


        output = model(src, trg)

        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]

        output_dim = output.shape[-1]

        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)

        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]

        loss = criterion(output, trg)

        optimizer.zero_grad()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

### Evaluation Function

In [39]:
def evaluate(model, iterator, criterion):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for i, batch in enumerate(iterator):

            src = batch[0].to(device)
            trg = batch[1].to(device)

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]

            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [40]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_loader, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')